In [1]:
!pip3 install sentence_transformers

In [2]:
!pip3 install pandas s3fs numpy 


In [3]:
!pip3 install faiss-cpu

In [5]:
import pandas as pd
import s3fs

import torch
from sentence_transformers import SentenceTransformer

import numpy as np
import pickle

#from vector_engine.utils import vector_search, id2details



In [12]:
import faiss

In [13]:
#df = pd.read_csv('s3://vector-search-blog/misinformation_papers.csv')

In [14]:
#!mkdir vector-search-blog
#df.to_csv('vector-search-blog/misinformation_papers.csv')

In [15]:
df = pd.read_csv('vector-search-blog/misinformation_papers.csv')

In [16]:
# Instantiate the sentence-level DistilBERT
model = SentenceTransformer('distilbert-base-nli-stsb-mean-tokens')

# Check if cuda is available and swithc to GPU
if torch.cuda.is_available():
    model = model.to(torch.device("cuda"))
print(model.device)

# Convert abstract to vectors
embeddings = model.encode(df.abstract.to_list(), show_progress_bar=True)

cpu


Batches:   0%|          | 0/264 [00:00<?, ?it/s]

In [17]:
dir(faiss)

['AdditiveQuantizer',
 'AlignedTableFloat32',
 'AlignedTableFloat32_round_capacity',
 'AlignedTableUint16',
 'AlignedTableUint16_round_capacity',
 'AlignedTableUint8',
 'AlignedTableUint8_round_capacity',
 'AlignedTable_to_array',
 'ArrayInvertedLists',
 'AutoTuneCriterion',
 'BitstringReader',
 'BitstringWriter',
 'BlockInvertedLists',
 'BufferList',
 'BufferedIOReader',
 'BufferedIOWriter',
 'ByteVector',
 'ByteVectorVector',
 'CMax_float_partition_fuzzy',
 'CMax_uint16_partition_fuzzy',
 'CMin_float_partition_fuzzy',
 'CMin_uint16_partition_fuzzy',
 'CenteringTransform',
 'CharVector',
 'Cloner',
 'Clustering',
 'ClusteringIterationStats',
 'ClusteringIterationStatsVector',
 'ClusteringParameters',
 'ConcatenatedInvertedLists',
 'DirectMap',
 'DirectMapAdd',
 'DistanceComputer',
 'DoubleVector',
 'EnumeratedVectors',
 'FAISS_VERSION_MAJOR',
 'FAISS_VERSION_MINOR',
 'FAISS_VERSION_PATCH',
 'FastScanStats',
 'FileIOReader',
 'FileIOWriter',
 'Float32Vector',
 'Float32VectorVector',
 '

In [19]:
# Step 1: change the data type
embeddings = np.array([embedding for embedding in embeddings]).astype("float32")

# Step 2: Instantiate the index
index = faiss.IndexFlatL2(embeddings.shape[1])

# Setp 3: Pass the index to IndexIDMap
index = faiss.IndexIDMap(index)

# Step 4: Add vector and their IDs
index.add_with_ids(embeddings, df.id.values)

# Retrive the 10 nearest neightbours
D, I = index.search(np.array([embeddings[5415]]), k=10)

In [24]:
D,I

(array([[ 0.       ,  1.3749892, 55.554802 , 65.87551  , 67.96593  ,
         69.05733  , 69.70645  , 70.40647  , 70.57283  , 71.03383  ]],
       dtype=float32),
 array([[3092618151, 3011345566, 3012936764, 3011186656, 3092128270,
         3048848247, 3044429417, 3055557295, 3024620668, 3044097955]]))

In [28]:
df.iloc[5415,1]

'The COVID-19 social media infodemic.'

In [30]:
def vector_search(query, model, index, num_results=10):
    """Tranforms query to vector using a pretrained, sentence-level
    DistilBERT model and finds similar vectors using FAISS.
    
    Args:
        query (str): User query that should be more than a sentence long.
        model (sentence_transformers.SentenceTransformer.SentenceTransformer)
        index (`numpy.ndarray`): FAISS index that needs to be deserialized.
        num_results (int): Number of results to return.
    
    Returns:
        D (:obj:`numpy.array` of `float`): Distance between results and query.
        I (:obj:`numpy.array` of `int`): Paper ID of the results.
    
    """
    vector = model.encode(list(query))
    D, I = index.search(np.array(vector).astype("float32"), k=num_results)
    return D, I


def id2details(df, I, column):
    """Returns the paper titles based on the paper index."""
    return [list(df[df.id == idx][column]) for idx in I[0]]


In [31]:
user_query = """
    WhatsApp was alleged to have been widely used to spread misinformation and propaganda 
    during the 2018 elections in Brazil and the 2019 elections in India. Due to the 
    private encrypted nature of the messages on WhatsApp, it is hard to track the dissemination 
    of misinformation at scale. In this work, using public WhatsApp data from Brazil and India, we 
    observe that misinformation has been largely shared on WhatsApp public groups even after they 
    were already fact-checked by popular fact-checking agencies. This represents a significant portion 
    of misinformation spread in both Brazil and India in the groups analyzed. We posit that such 
    misinformation content could be prevented if WhatsApp had a means to flag already fact-checked 
    content. To this end, we propose an architecture that could be implemented by WhatsApp to counter 
    such misinformation. Our proposal respects the current end-to-end encryption architecture on WhatsApp,
    thus protecting users’ privacy while providing an approach to detect the misinformation that benefits 
    from fact-checking efforts.
"""

In [32]:

# Querying the index
D, I = vector_search([user_query], model, index, num_results=10)

In [33]:
id2details(df, I, 'original_title')

[['Can WhatsApp Benefit from Debunked Fact-Checked Stories to Reduce Misinformation?'],
 ['A Dataset of Fact-Checked Images Shared on WhatsApp During the Brazilian and Indian Elections'],
 ['A Dataset of Fact-Checked Images Shared on WhatsApp During the Brazilian and Indian Elections'],
 ['A System for Monitoring Public Political Groups in WhatsApp'],
 ['Politics of Fake News: How WhatsApp Became a Potent Propaganda Tool in India'],
 ['Characterizing Attention Cascades in WhatsApp Groups'],
 ['Can WhatsApp Counter Misinformation by Limiting Message Forwarding'],
 ['Can WhatsApp Counter Misinformation by Limiting Message Forwarding'],
 ['OS IMPACTOS JURÍDICOS E SOCIAIS DAS FAKE NEWS EM TERRITÓRIO BRASILEIRO'],
 ['Images and Misinformation in Political Groups: Evidence from WhatsApp in India']]